In [330]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [331]:
train_data = pd.read_csv('/Users/eashan22/Desktop/DataMining/FinalProject/Dataset/train_data.csv',sep=',')
train_label = pd.read_csv('/Users/eashan22/Desktop/DataMining/FinalProject/Dataset/train_label.csv',sep=',')
test_data = pd.read_csv('/Users/eashan22/Desktop/DataMining/FinalProject/Dataset/test_data.csv',sep=',')

In [332]:
train_data.replace(regex={'<.*?>|&nbsp|\W':' '}, inplace=True)

In [333]:
train_lvl2 = train_data[train_data.columns[3]] # train_X11
train_lvl3 = train_data[train_data.columns[4]] # train_X12
train_lvl23 = pd.concat([train_lvl2, train_lvl3], axis=1)  # train_X13

train_XX = train_lvl23.fillna(axis=1,method='ffill')
train_X1 = train_XX[train_XX.columns[1]]

#train_X1=train_data[train_data.columns[4]]

In [334]:
train_price = train_data[train_data.columns[6]] #train_X22
train_type = train_data[train_data.columns[7]] #train_X3
train_y = train_label[train_label.columns[1:]]

#train_X1=train_X1.dropna()
# train_y.shape

In [335]:
train_X1 = pd.get_dummies(train_X1)
train_P = (train_price - train_price.min()) / (train_price.max() - train_price.min())  #train_X2
train_type = pd.get_dummies(train_type)

train_X = pd.concat([train_X1, train_P,train_type], axis=1)
train_y = np.ravel(train_y)

print(train_X.shape)

(18141, 194)


In [337]:
# from numpy import array
# from numpy import argmax
# from sklearn.preprocessing import LabelEncoder

# # integer encode
# label_encoder = LabelEncoder()
# price_encoded = label_encoder.fit_transform(train_X.price)

# # onehot_encoder = OneHotEncoder()
# # integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# # onehot_encoded = onehot_encoder.fit_transform(train_X)
# # onehot_encoded = pd.DataFrame(onehot_encoded)

# price_encoded = pd.DataFrame(price_encoded)
# train_X = train_X.append(price_encoded)
# train_X.index = train_X.index-18142
# train_X = train_X.fillna(0)
# print(train_X)

In [ ]:
test_lvl2 = test_data[test_data.columns[3]]
test_lvl3 = test_data[test_data.columns[4]]
test_lvl23 = pd.concat([test_lvl2, test_lvl3], axis=1)

test_XX = test_lvl23.fillna(axis=1,method='ffill')
test_X1 = test_XX[test_XX.columns[1]]

test_price = test_data[test_data.columns[6]]
test_type = test_data[test_data.columns[7]]
test_X1 = pd.get_dummies(test_X1)
test_P = (test_price - test_price.min()) / (test_price.max() - test_price.min())
test_type = pd.get_dummies(test_type)
test_X4 = test_data[test_data.columns[0]]

test_X = pd.concat([test_X1, test_P, test_type], axis=1)

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
label_encoder = LabelEncoder()
price_encoded = label_encoder.fit_transform(test_X.price)
# print(integer_encoded)

price_encoded = pd.DataFrame(price_encoded)
test_X = test_X.append(price_encoded)

test_X = test_X.fillna(method='pad', axis=1)
print(test_X.shape)
# test_X = test_X.drop([x for x in range(6579,6603)], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = LogisticRegression()
model.fit(train_X,train_y)
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=3)),
                    ('clf', LogisticRegression())])
pipe_lr.fit(train_X, train_y)
y_pred = model.predict_proba(test_X)

from sklearn.model_selection import GridSearchCV
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'clf__C': param_range, }]

gs = GridSearchCV(estimator=pipe_lr, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=13,
                  n_jobs=-1)

gs = gs.fit(train_X, train_y)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(train_X, train_y)

from sklearn.model_selection import cross_val_score
gs = GridSearchCV(estimator=pipe_lr,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=13)

# Note: Optionally, you could use cv=2 
# in the GridSearchCV above to produce
# the 5 x 2 nested CV that is shown in the figure.

scores = cross_val_score(gs, train_X, train_y, scoring='accuracy', cv=13)

In [ ]:
Y_pred = pd.DataFrame(y_pred, columns=[0,'score'])
Y_pred.index = Y_pred.index
Y_pred = Y_pred.drop(0,axis=1)
result = pd.concat([test_data['id'],Y_pred], axis=1)

print("model.coef_: {}".format(model.coef_))
print("model.intercept_: {}".format(model.intercept_))
print("Training set score: {:.2f}".format(model.score(train_X, train_y)))

In [ ]:
result
# result.to_csv("/Users/eashan22/Desktop/DataMining/FinalProject/Dataset/results/LR_prl_20_.csv",sep=',',index=0,float_format='%.8f')